In [15]:
import pandas as pd
import os
import git

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        #git_root = git_repo.git.rev_parse("--show-toplevel")
        
        return git_repo.working_dir

In [20]:
top_level_git_dir = get_git_root(os.getcwd())
raw_data_dir = os.path.join(top_level_git_dir, "data", "raw")

train_csv_path = os.path.join(raw_data_dir, "train_values.csv")
test_csv_path = os.path.join(raw_data_dir, "test_values.csv")
train_labels_csv_path = os.path.join(raw_data_dir, "train_labels.csv")
submission_format_csv_path = os.path.join(raw_data_dir, "submission_format.csv")

train_df = pd.read_csv(train_csv_path, index_col = "row_id")
train_labels_df = pd.read_csv(train_labels_csv_path, index_col = "process_id")
test_df = pd.read_csv(test_csv_path, index_col = "row_id")
submission_format_csv_path = pd.read_csv(submission_format_csv_path, index_col = "process_id")


/Users/greg.bolla/.local/share/virtualenvs/driven-data-rinse-over-run-w5STMFPe/lib/python3.6/site-packages/numpy/lib/arraysetops.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [39]:
train_df.columns

Index(['process_id', 'object_id', 'phase', 'timestamp', 'pipeline',
       'supply_flow', 'supply_pressure', 'return_temperature',
       'return_conductivity', 'return_turbidity', 'return_flow', 'supply_pump',
       'supply_pre_rinse', 'supply_caustic', 'return_caustic', 'supply_acid',
       'return_acid', 'supply_clean_water', 'return_recovery_water',
       'return_drain', 'object_low_level', 'tank_level_pre_rinse',
       'tank_level_caustic', 'tank_level_acid', 'tank_level_clean_water',
       'tank_temperature_pre_rinse', 'tank_temperature_caustic',
       'tank_temperature_acid', 'tank_concentration_caustic',
       'tank_concentration_acid', 'tank_lsh_caustic', 'tank_lsh_acid',
       'tank_lsh_clean_water', 'tank_lsh_pre_rinse', 'target_time_period'],
      dtype='object')

In [38]:
train_df.head()
proc_id = 20001
proc_id_df = train_df[train_df.process_id == proc_id]
proc_id_df[proc_id_df.phase == "final_rinse"]

,process_id,object_id,phase,timestamp,pipeline,supply_flow,supply_pressure,return_temperature,return_conductivity,return_turbidity,...,tank_temperature_pre_rinse,tank_temperature_caustic,tank_temperature_acid,tank_concentration_caustic,tank_concentration_acid,tank_lsh_caustic,tank_lsh_acid,tank_lsh_clean_water,tank_lsh_pre_rinse,target_time_period
row_id,,,,,,,,,,,,,,,,,,,,,
1425,20001,405,final_rinse,2018-04-15 05:08:17,L4,12684.462000,-0.035807,72.424770,42.875420,0.564236,...,33.604603,82.852290,72.80093,45.843548,44.653038,False,0.0,False,0.0,False
1426,20001,405,final_rinse,2018-04-15 05:08:19,L4,6210.214400,-0.035590,72.424770,42.961740,0.636574,...,33.604603,82.852290,72.80093,45.839340,44.653038,False,0.0,False,0.0,False
1427,20001,405,final_rinse,2018-04-15 05:08:21,L4,3349.247600,-0.035807,72.424770,42.023487,0.408709,...,33.600983,82.848670,72.80093,45.838448,44.653038,False,0.0,False,0.0,False
1428,20001,405,final_rinse,2018-04-15 05:08:23,L4,1634.837900,-0.035807,72.424770,42.762917,0.466580,...,33.600983,82.870370,72.80093,45.838734,44.653038,False,0.0,False,0.0,False
1429,20001,405,final_rinse,2018-04-15 05:08:25,L4,799.334530,-0.035807,72.403070,42.337997,0.886140,...,33.604603,82.823350,72.80093,45.838120,44.653038,False,0.0,False,0.0,False
1430,20001,405,final_rinse,2018-04-15 05:08:27,L4,437.644680,-0.035807,72.403070,37.117530,0.853588,...,33.604603,82.823350,72.80093,45.841644,44.653038,False,0.0,False,0.0,False
1431,20001,405,final_rinse,2018-04-15 05:08:29,L4,242.332180,-0.035807,72.312645,37.155860,0.813802,...,33.582897,82.848670,72.80093,45.840332,44.653038,False,0.0,False,0.0,False
1432,20001,405,final_rinse,2018-04-15 05:08:31,L4,122.974530,-0.035807,72.196910,34.016205,1.128472,...,33.582897,82.870370,72.80093,45.841194,44.653038,False,0.0,False,0.0,False
1433,20001,405,final_rinse,2018-04-15 05:08:33,L4,72.337960,-0.035807,72.073930,32.861076,0.636574,...,33.582897,82.848670,72.80093,45.836502,44.653038,False,0.0,False,0.0,False


,process_id,object_id,phase,timestamp,pipeline,supply_flow,supply_pressure,return_temperature,return_conductivity,return_turbidity,...,tank_temperature_pre_rinse,tank_temperature_caustic,tank_temperature_acid,tank_concentration_caustic,tank_concentration_acid,tank_lsh_caustic,tank_lsh_acid,tank_lsh_clean_water,tank_lsh_pre_rinse,target_time_period
row_id,,,,,,,,,,,,,,,,,,,,,
0,20000,427,pre_rinse,2018-04-30 21:39:21,L4,17039.207,0.480035,13.726129,0.337567,0.018084,...,29.922598,82.94994,72.526050,45.378080,45.124700,False,0.0,False,0.0,False
1,20000,427,pre_rinse,2018-04-30 21:39:23,L4,29390.912,0.554253,13.852720,0.335876,0.025318,...,29.922598,83.01143,72.526050,45.385216,45.125390,False,0.0,False,0.0,False
2,20000,427,pre_rinse,2018-04-30 21:39:25,L4,24323.640,0.657118,13.939526,0.335706,0.025318,...,29.922598,83.07292,72.529655,45.383460,45.125343,False,0.0,False,0.0,False
3,20000,427,pre_rinse,2018-04-30 21:39:27,L4,17180.266,0.749132,14.001014,0.335571,0.025318,...,29.944302,83.07292,72.526050,45.375385,45.125360,False,0.0,False,0.0,False
4,20000,427,pre_rinse,2018-04-30 21:39:29,L4,11754.919,0.795139,14.001014,0.335509,0.028935,...,29.922598,83.08739,72.529655,45.374237,45.121326,False,0.0,False,0.0,False


In [23]:
train_labels_df.head()

,final_rinse_total_turbidity_liter
process_id,
20001,4.318275e+06
20002,4.375286e+05
20003,4.271977e+05
20004,7.197830e+05
20005,4.133107e+05
